In [16]:
import pandas as pd    
import numpy as np
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)

import matplotlib.pyplot as plt

In [17]:
df_2010_2011 = pd.read_excel("online_retail_II.xlsx", sheet_name = "Year 2010-2011")
df = df_2010_2011.copy()
online_retail_df = pd.read_excel("online_retail_II.xlsx", sheet_name = "Year 2010-2011")
today_date = dt.datetime(2011,12,30)


In [18]:
returned = df[df["Invoice"].str.contains("C",na=False)].index
df = df.drop(returned, axis = 0)



In [19]:
df["TotalPrice"] = df["Quantity"]*df["Price"]

df.dropna(inplace = True)

In [20]:
#RECENCY METRIC
import datetime as dt
today_date = dt.datetime(2011,12,9)
df["Customer ID"] = df["Customer ID"].astype(int)
temp_df = (today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"}))
temp_df.rename(columns={"InvoiceDate": "Recency"}, inplace = True)
recency_df = temp_df["Recency"].apply(lambda x: x.days)

# FREQUENCY METRIC
temp_df = df.groupby(["Customer ID","Invoice"]).agg({"Invoice":"count"})
freq_df = temp_df.groupby("Customer ID").agg({"Invoice":"count"})
freq_df.rename(columns={"Invoice": "Frequency"}, inplace = True)

# MONETARY METRIC
monetary_df = df.groupby("Customer ID").agg({"TotalPrice":"sum"})
monetary_df.rename(columns = {"TotalPrice": "Monetary"}, inplace = True)
rfm = pd.concat([recency_df, freq_df, monetary_df],  axis=1)
df = rfm
df["RecencyScore"] = pd.qcut(df['Recency'], 5, labels = [5, 4, 3, 2, 1])
df["FrequencyScore"] = pd.qcut(df['Frequency'].rank(method = "first"), 5, labels = [1,2,3,4,5])
df["MonetaryScore"] = pd.qcut(df['Monetary'], 5, labels = [1,2,3,4,5])
df["RFM_SCORE"] = df['RecencyScore'].astype(str) + df['FrequencyScore'].astype(str) + df['MonetaryScore'].astype(str)
seg_map = {
        r'[1-2][1-2]': 'Hibernating',
        r'[1-2][3-4]': 'At Risk',
        r'[1-2]5': 'Can\'t Loose',
        r'3[1-2]': 'About to Sleep',
        r'33': 'Need Attention',
        r'[3-4][4-5]': 'Loyal Customers',
        r'41': 'Promising',
        r'51': 'New Customers',
        r'[4-5][2-3]': 'Potential Loyalists',
        r'5[4-5]': 'Champions'
}

df['Segment'] = df['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
df['Segment'] = df['Segment'].replace(seg_map, regex=True)
df.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346,324,1,77183.60,1,1,5,115,Hibernating
12347,1,7,4310.00,5,5,5,555,Champions
12348,74,4,1797.24,2,4,4,244,At Risk
12349,17,1,1757.55,4,1,4,414,Promising
12350,309,1,334.40,1,1,2,112,Hibernating


In [23]:
rfm = df.loc[:,"Recency":"Monetary"]

In [24]:
df.groupby("Customer ID").agg({"Segment": "sum"}).head()

,Segment
Customer ID,
12346,Hibernating
12347,Champions
12348,At Risk
12349,Promising
12350,Hibernating


In [26]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler((0,1))
df = sc.fit_transform(rfm)
kmeans = KMeans(n_clusters = 10)
k_fit = kmeans.fit(df)
k_fit.labels_

array([7, 5, 9, ..., 5, 4, 1], dtype=int32)

In [27]:
kmeans = KMeans(n_clusters = 6).fit(df)
kumeler = kmeans.labels_
pd.DataFrame({"Customer ID": rfm.index, "Kumeler": kumeler})
rfm["cluster_no"] = kumeler
rfm["cluster_no"] = rfm["cluster_no"] + 1

In [28]:
rfm.groupby("cluster_no").agg({"cluster_no":"count"})

,cluster_no
cluster_no,
1,2053
2,441
3,1028
4,500
5,24
6,293


In [29]:
rfm.head()

,Recency,Frequency,Monetary,cluster_no
Customer ID,,,,
12346,324,1,77183.60,6
12347,1,7,4310.00,1
12348,74,4,1797.24,3
12349,17,1,1757.55,1
12350,309,1,334.40,6
